In [7]:
from PyUoI.UoINMF import UoINMF
from activ import load_data, data_normalization
import os
from activ.readfile import TrackTBIFile
%matplotlib inline
import matplotlib.pyplot as plt
import h5py
import numpy as np
from activ.analytics import heatmap, nmf_bases_heatmap
from activ.clustering import UmapClusteringResults
from os.path import dirname, join
from scipy.optimize import curve_fit
from scipy.stats import norm, sem
from matplotlib.backends.backend_pdf import PdfPages
from IPython.display import display, Latex, Markdown
from pylatex import Document, Section, Subsection, Command
from pylatex import Document, Section, Figure, SubFigure, NoEscape
from pylatex.utils import italic, NoEscape
from scipy.stats import norm, sem

In [8]:
nmfdata = load_data(uoinmf=True)
orig = load_data(uoinmf=False)
origoc = orig.outcomes
feature_names = nmfdata.biomarker_features
bmbases = nmfdata.biomarker_bases
bmbases.shape

(6, 235)

In [9]:
def drop_x_percent(A,h,feature_names,percent):
    total_features = h.shape[1]
    num_features_to_drop = np.round(total_features * percent//100)
    h_sum = np.sum(h, axis=0)
    h_sum_sorted = np.argsort(h_sum)[::-1]
    indices_highest_values = h_sum_sorted[:num_features_to_drop]
    dropped_features = sorted(feature_names[indices_highest_values])
    h_new = np.delete(h, indices_highest_values, axis=1)
    A_new = np.delete(A, indices_highest_values, axis=1)    
    return A_new, h_new, dropped_features

In [10]:
def get_biomarker_transformed_weight(dropped_bases,A):
    uoinmf = UoINMF()
    uoinmf.components_ = dropped_bases
    data = data_normalization.data_normalization(A, 'positive')
    W = uoinmf.transform(data)
    return W

In [11]:
percent = [0,10,20,30,40,50,60,70,80,90]
for p in percent:
    A_features_dropped, features_dropped, dropped_feature_names = drop_x_percent(orig.biomarkers,bmbases,feature_names,p)
    w = get_biomarker_transformed_weight(features_dropped, A_features_dropped)
    print(A_features_dropped.shape, features_dropped.shape, w.shape)
    fname = '/Users/ahyeon/data/activ/bm_features_dropped/bm_features_dropped_opp_{}.h5'.format(p)
    TrackTBIFile.write(fname, w, origoc)
    

(247, 235) (6, 235) (247, 6)
(247, 212) (6, 212) (247, 6)
(247, 188) (6, 188) (247, 6)
(247, 165) (6, 165) (247, 6)
(247, 141) (6, 141) (247, 6)
(247, 118) (6, 118) (247, 6)
(247, 94) (6, 94) (247, 6)
(247, 71) (6, 71) (247, 6)
(247, 47) (6, 47) (247, 6)
(247, 24) (6, 24) (247, 6)
